In [1]:
import sys
sys.path.insert(0, "../..")

In [2]:
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
import numpy as np
from copy import deepcopy

from nn_extrapolation import AcceleratedSGD

In [3]:
torch.cuda.is_available()

True

In [4]:
val_loss_fn = nn.NLLLoss(reduction="sum")

def validation(model, loader):
    ok = 0
    loss_sum = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.cuda()
            y = y.cuda()
            out = model(x)
            loss_sum += val_loss_fn(out, y)
            preds = out.argmax(1)
            ok += (y == preds).sum()
            total += len(y)
    return ok / total, loss_sum / total

def train_epoch(loss_log):
    model.train()
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
        optimizer.zero_grad()
        out = model(x)
        loss = loss_fn(out, y)
        loss_log += list(loss.flatten().cpu().detach().numpy())
        loss.backward()
        optimizer.step()

In [5]:
train_ds = datasets.MNIST("../../../MNIST", download=True, train=True, transform=transforms.ToTensor())
test_ds = datasets.MNIST("../../../MNIST", download=True, train=False, transform=transforms.ToTensor())
valid_size = int(0.2 * len(train_ds))
train_ds, valid_ds = data.random_split(train_ds, [len(train_ds) - valid_size, valid_size])

train_loader = data.DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=2)
valid_loader = data.DataLoader(valid_ds, batch_size=64, shuffle=True, num_workers=2)
test_loader = data.DataLoader(test_ds, batch_size=64, shuffle=False, num_workers=2)

In [6]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [7]:
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.8, weight_decay=1e-5, mode="epoch")
loss_fn = nn.NLLLoss()

In [8]:
log_file = open("SGD_momentum.txt", "w")

## Epoch

In [9]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.3505
Validation accuracy: 0.8257, validation loss: 0.9096
Epoch 2
Training loss: 0.7708
Validation accuracy: 0.8518, validation loss: 0.6832
Epoch 3
Training loss: 0.6259
Validation accuracy: 0.8621, validation loss: 0.5911
Epoch 4
Training loss: 0.5560
Validation accuracy: 0.8692, validation loss: 0.5398
Epoch 5
Training loss: 0.5134
Validation accuracy: 0.8733, validation loss: 0.5060
Epoch 6
Training loss: 0.4842
Validation accuracy: 0.8758, validation loss: 0.4820
Epoch 7
Training loss: 0.4626
Validation accuracy: 0.8797, validation loss: 0.4640
Epoch 8
Training loss: 0.4457
Validation accuracy: 0.8825, validation loss: 0.4497
Epoch 9
Training loss: 0.4322
Validation accuracy: 0.8836, validation loss: 0.4380
Epoch 10
Training loss: 0.4210
Validation accuracy: 0.8853, validation loss: 0.4284
Epoch 11
Training loss: 0.4115
Validation accuracy: 0.8866, validation loss: 0.4203
Epoch 12
Training loss: 0.4033
Validation accuracy: 0.8881, validation loss: 0.4132
E

In [10]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9069, device='cuda:0'), tensor(0.3371, device='cuda:0'))
Valid: (tensor(0.9007, device='cuda:0'), tensor(0.3572, device='cuda:0'))


In [11]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [12]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [13]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9144, device='cuda:0'), tensor(0.3070, device='cuda:0'))
Valid: (tensor(0.9064, device='cuda:0'), tensor(0.3332, device='cuda:0'))


In [14]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [15]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [16]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9144, device='cuda:0'), tensor(0.3070, device='cuda:0'))
Valid: (tensor(0.9064, device='cuda:0'), tensor(0.3332, device='cuda:0'))


## Epoch average

In [17]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(28*28, 10),
    nn.LogSoftmax(-1),
)
model.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [18]:
log_file = open("SGD_momentum-avg.txt", "w")
optimizer = AcceleratedSGD(model.parameters(), 1e-3, k=10, momentum=0.8, weight_decay=1e-5, mode="epoch_avg")

In [19]:
epochs = 30

for epoch in range(epochs):
    print("Epoch", epoch+1)
    loss_log = []
    train_epoch(loss_log)
    print(f"Training loss: {np.mean(loss_log):.4f}")
    optimizer.finish_epoch()
    val_acc, val_loss = validation(model, valid_loader)
    print(f"Validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}")
    print("Epoch", epoch+1, 
          f"Training loss: {np.mean(loss_log):.4f}, validation accuracy: {val_acc:.4f}, validation loss: {val_loss:.4f}",
          file=log_file, flush=True
         )

Epoch 1
Training loss: 1.3505
Validation accuracy: 0.8313, validation loss: 0.9120
Epoch 2
Training loss: 0.7718
Validation accuracy: 0.8523, validation loss: 0.6838
Epoch 3
Training loss: 0.6259
Validation accuracy: 0.8621, validation loss: 0.5914
Epoch 4
Training loss: 0.5557
Validation accuracy: 0.8695, validation loss: 0.5394
Epoch 5
Training loss: 0.5130
Validation accuracy: 0.8735, validation loss: 0.5057
Epoch 6
Training loss: 0.4836
Validation accuracy: 0.8775, validation loss: 0.4817
Epoch 7
Training loss: 0.4620
Validation accuracy: 0.8792, validation loss: 0.4634
Epoch 8
Training loss: 0.4452
Validation accuracy: 0.8823, validation loss: 0.4493
Epoch 9
Training loss: 0.4316
Validation accuracy: 0.8834, validation loss: 0.4376
Epoch 10
Training loss: 0.4203
Validation accuracy: 0.8846, validation loss: 0.4279
Epoch 11
Training loss: 0.4109
Validation accuracy: 0.8871, validation loss: 0.4198
Epoch 12
Training loss: 0.4027
Validation accuracy: 0.8882, validation loss: 0.4127
E

In [20]:
train_score = validation(model, train_loader)
valid_score = validation(model, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9069, device='cuda:0'), tensor(0.3367, device='cuda:0'))
Valid: (tensor(0.8997, device='cuda:0'), tensor(0.3568, device='cuda:0'))


In [21]:
optimizer.param_groups[0]["method"] = "RNA"
optimizer.accelerate()

In [22]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [23]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9079, device='cuda:0'), tensor(0.3312, device='cuda:0'))
Valid: (tensor(0.9009, device='cuda:0'), tensor(0.3520, device='cuda:0'))


In [24]:
optimizer.param_groups[0]["method"] = "RRE"
optimizer.accelerate()

In [25]:
model_acc = deepcopy(model)
optimizer.store_parameters([model_acc.parameters()])
model_acc.cuda()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=10, bias=True)
  (2): LogSoftmax(dim=-1)
)

In [26]:
train_score = validation(model_acc, train_loader)
valid_score = validation(model_acc, valid_loader)
print("Train:", train_score)
print("Valid:", valid_score)
print("Train:", train_score, flush=True, file=log_file)
print("Valid:", valid_score, flush=True, file=log_file)

Train: (tensor(0.9079, device='cuda:0'), tensor(0.3312, device='cuda:0'))
Valid: (tensor(0.9009, device='cuda:0'), tensor(0.3520, device='cuda:0'))
